In [2]:
from sympy import Symbol, Implies, And, Or, Not
from sympy.logic.boolalg import truth_table
from sympy.solvers import solve

import json
import numpy as np

from pathlib import Path

In [23]:
# feature type
BOOL = 'bool'
INT = 'int'
REAL = 'real'

OPTIONAL = "optional"
MANDATORY = "mandatory"

NO_GROUP = "no_group"
OR = "or"
ALTERNATIVE = "alternative"

REQUIRES = "requires"
EXCLUDES = "excludes"

ROOT = "root"
SYSTEM = "system"
CONTEXT = "context"
STRUCTURE = "structure"
CROSS_TREE_CONSTRAINTS = "cross tree constraints"
INTERVAL_SIZE = "interval size"

In [24]:
class Feature:
    """
    Class used to represent a Feature
    """
    

    def __init__(self, feature: list[str, str, float, float, str], branch: str, interval_size: int = 1) -> None:
        """
        feature: list
            a list of feature attributes [name, type, lb, ub, optional]
        interval_size: float
            the size of intervals for discretization of numerical features (INT, REAL)
        branch: str
            SYSTEM, CONTEXT, None
        """
        self.name = feature[0]
        self.type = feature[1]
        self.lb = feature[2]
        self.ub = feature[3]
        if feature[4] == OPTIONAL:
            self.optional = True
        elif feature[4] == MANDATORY:
            self.optional = False
        else:
            raise ValueError('Invalid optional / mandatory definition', feature)
        self.interval_size = interval_size
        self.branch = branch

        self.parent = None
        self.group_flag = False
        self.symbol = Symbol(self.name)

class Structure:
    """
    Class used to represent a Feature Structure (Alternative-, or-, no-group)
    """

    def __init__(self, parent: str, children: list[str], type: str) -> None:
        """
        parent: Feature
            the parent feature
        children: list of Feature
            list of child features
        relationship: str
            NO_GROUP, ALTERNATIVE, OR
        """
        self.parent = parent
        self.children = children
        self.type = type

class CrossTreeConstraint:
    """
    Class used to represent a Cross-Tree Constraint (requires, excludes)
    """


    def __init__(self, feature_1: str, feature_2: str, constraint: str) -> None:
        """
        feature_1: Feature
            first feature of the constraint
        feature_2: Feature
            second feature of the constraint
        constraint: str
            REQUIRES, EXCLUDES
        """
        self.feature_1 = feature_1
        self.feature_2 = feature_2
        self.constraint = constraint

class NumericalSubFeature():

    def __init__(self, name: str, branch: str, lb: float, ub: float) -> None:
        self.name = name
        self.branch = branch
        self.lb = lb
        self.ub = ub

In [39]:
class FM:

    def __init__(self, json_file):

        with open(json_file) as file:
            fm_json = json.load(file)
        
        self.features = {}
        self.numerical_sub_features = {}
        self.features[ROOT] = Feature([ROOT, BOOL, 1, 1, MANDATORY], None)
        self.features[SYSTEM] = Feature([SYSTEM, BOOL, 1, 1, MANDATORY], None)
        self.features[SYSTEM].parent = self.features[ROOT]
        self.features[CONTEXT] = Feature([CONTEXT, BOOL, 1, 1, MANDATORY], None)
        self.features[CONTEXT].parent = self.features[ROOT]

        for feature in fm_json[SYSTEM]:
            feature_obj = Feature(feature, SYSTEM)
            self.features[feature_obj.name] = feature_obj
        for feature in fm_json[CONTEXT]:
            feature_obj = Feature(feature, CONTEXT)
            self.features[feature_obj.name] = feature_obj
        for _, feature in self.features.items():
            try:
                feature.interval_size = fm_json[INTERVAL_SIZE][feature.name]
            except KeyError:
                pass
        
        self.structures = []
        for structure in fm_json[STRUCTURE]:
            child_features = []
            group_type = structure[2]
            for child_name in structure[1]:
                child_features.append(self.features[child_name])
                if group_type != NO_GROUP:
                    self.features[child_name].group_flag = True
                self.features[child_name].parent = self.features[structure[0]]
            self.structures.append(Structure(self.features[structure[0]], child_features, group_type))

        self.cross_tree_constraints = []
        for constraint in fm_json[CROSS_TREE_CONSTRAINTS]:
            self.cross_tree_constraints.append(self.features[constraint[0]], self.features[constraint[1]], constraint[2])

        self.fm_pl = And(Implies(True, self.features[ROOT].symbol))
        self.create_feature_relationships()

        self.ordered_names = self.generate_numerical_truth_table()[1]

        self.context_ordered_names = []
        for name in self.ordered_names:
            if self.get_feature_branch(name) == CONTEXT:
                self.context_ordered_names.append(name)
    
        self.system_ordered_names = []
        for name in self.ordered_names:
            if self.get_feature_branch(name) == SYSTEM:
                self.system_ordered_names.append(name)        
        
    def add_pl_term(self, term):
        self.fm_pl = And(self.fm_pl, term)
    
    def create_feature_relationships(self):
        for feature in self.features.values():
            if not feature.group_flag and feature.parent != None:
                # feature is not part of a group
                self.add_pl_term(Implies(feature.symbol, feature.parent.symbol))
                if not feature.optional:
                    # feature is mandatory
                    self.add_pl_term(Implies(feature.parent.symbol, feature.symbol))

        for structure in self.structures:
            if structure.type == OR:
                self.add_pl_term(And(Implies(structure.parent.symbol, Or(*[child.symbol for child in structure.children])),
                                     Implies(Or(*[child.symbol for child in structure.children]), structure.parent.symbol)))
            elif structure.type == ALTERNATIVE:
                for child in structure.children:
                    conjunctive_terms = [structure.parent.symbol]
                    for other_child in structure.children:
                        if child.name != other_child.name:
                            conjunctive_terms.append(Not(other_child.symbol))

                    self.add_pl_term(And(Implies(child.symbol, And(*conjunctive_terms)),
                                         Implies(And(*conjunctive_terms), child.symbol)))
            elif structure.type == NO_GROUP:
                pass
            else:
                raise ValueError('Invalid structure type', structure.parent.name, structure.type)
        
    def generate_truth_table(self):
        fm_file_path = 'fm_saves/fm_valid_configs.json'
        fm_file = Path(fm_file_path)

        if fm_file.is_file():
            with open(fm_file_path, 'r') as f:
                valid_table = json.load(f)
        else:
            valid_table = []
            table = truth_table(self.fm_pl, [feature.symbol for feature in self.features.values()])
            for line in table:
                if line[-1]:
                    valid_table.append(line[0])
            with open(fm_file_path, 'w') as f:
                json.dump(valid_table, f)
        
        return valid_table
    
    def get_ordered_names(self):
        return list(self.features.keys())
    
    def generate_numerical_truth_table(self):
        valid_table = self.generate_truth_table()

        ordered_names = list(self.features.keys())
        # Handle numerical features
        for parent_feature_index, feature in enumerate(self.features.values()):
            if feature.type == INT or feature.type == REAL:
                
                range_lb = feature.lb
                range_ub = feature.ub
                range_interval_size = feature.interval_size

                if feature.type == REAL:
                    range_ub = int((range_ub - range_lb) / range_interval_size)
                    range_lb = 0
                    range_interval_size = 1
                else: 
                    range_ub += 1

                numerical_sub_feature_list = []
                for index_match in range(range_lb, range_ub, range_interval_size):
                    sub_feature_name = feature.name + '_' + str(index_match)
                    ordered_names.append(sub_feature_name)
                    
                    numerical_sub_feature_list.append(NumericalSubFeature(sub_feature_name, 
                                                                          feature.branch,
                                                                          feature.lb + feature.interval_size * index_match, 
                                                                          feature.lb + (index_match + 1) * feature.interval_size))

                self.numerical_sub_features[feature.name] = numerical_sub_feature_list

                temp_valid_table = []
                for entry in valid_table:
                    # check if feature is part of valid config
                    if entry[parent_feature_index] == 1:

                        for index_match in range(range_lb, range_ub, range_interval_size):
                            temp_valid_table.append(entry + [1 if index == index_match else 0 for index in range(range_lb, range_ub, range_interval_size)])
                    else:
                        temp_valid_table.append(entry + [0 for _ in range(range_lb, range_ub, range_interval_size)])

                    valid_table = temp_valid_table

        return valid_table, ordered_names
    
    def numerical_feature_name_to_value_range(self, numerical_feature_name):
        for numerical_features in self.numerical_sub_features.values():
            for sub_feature in numerical_features:
                if sub_feature.name == numerical_feature_name:
                    return (sub_feature.lb, sub_feature.ub)
        raise ValueError('Numerical sub feature does not exist', numerical_feature_name)

    def numerical_feature_value_to_numerical_name(self, feature_name, value):
        last_feature = None
        for sub_feature in self.numerical_sub_features[feature_name]:
            last_feature = sub_feature
            if value >= sub_feature.lb and value < sub_feature.ub:
                return sub_feature.name
        if last_feature.ub == value and self.features[feature_name].type == REAL:
            return last_feature.name
        raise ValueError('Value outside of valid range', 'feature: {}, value: {}'.format(feature_name, value))                
        
    def get_feature_branch(self, feature_name):
        sub_features = {}
        for features in self.numerical_sub_features.values():
            for feature in features:
                sub_features[feature.name] = feature
        all_features = self.features | sub_features
        return all_features[feature_name].branch
    
    def get_system_mask(self, ordered_names):
        system_mask = [True if self.get_feature_branch(name) == SYSTEM else False for name in ordered_names]
        return system_mask
    
    def get_context_mask(self, ordered_names):
        context_mask = [True if self.get_feature_branch(name) == CONTEXT else False for name in ordered_names]
        return context_mask
    
    def get_context_ordered_names(self, ordered_names):
        context_ordered_names = []
        for name in ordered_names:
            if self.get_feature_branch(name) == CONTEXT:
                context_ordered_names.append(name)
        return context_ordered_names
    
    def translate_binary_context_to_str(self, context):
        return ''.join([str(i) for i in context])

    def translate_str_context_to_binary(self, context):
        return [int(i) for i in context]
    
    def get_context_system_dictionary(self):
        truth_table, ordered_names = self.generate_numerical_truth_table()
        context_dict = {}
        for config in truth_table:
            system = []
            context = []
            for feature, system_mask, context_mask in zip(config, self.get_system_mask(ordered_names), self.get_context_mask(ordered_names)):
                if system_mask:
                    system.append(feature)                
                if context_mask:
                    context.append(feature)

            str_context = self.translate_binary_context_to_str(context)

            if str_context in context_dict:
                context_dict[str_context].append(system)
            else:
                context_dict[str_context] = [system]
            
        return context_dict
    
    def get_extended_context_system_dictionary(self):
        truth_table, ordered_names = self.generate_numerical_truth_table()
        context_dict = {}
        for config in truth_table:
            system = []
            context = []
            for feature, system_mask, context_mask in zip(config, self.get_system_mask(ordered_names), self.get_context_mask(ordered_names)):
                if system_mask:
                    system.append(feature)                
                if context_mask:
                    context.append(feature)

            str_context = self.translate_binary_context_to_str(context)

            if str_context in context_dict:
                context_dict[str_context].append(system)
            else:
                context_dict[str_context] = [system]

        return context_dict
    
    # TODO
    # Create fixed system names order and context names order
    # Create valid configuration list
    # get context from whole config
    # get system from whole config

    # In Context System dict, make context tuple (A, B, C, D)
    # Make System config: {"feature_0": 1, "feature_1": 0, "feature_2": 1...}
    # context system dict: dict(tuple: List(dict))
    

In [40]:
if False:
    feature_model = FM('swim_fm.json')
    context_system_dict = feature_model.get_context_system_dictionary()
    print(context_system_dict)
    print(feature_model.generate_numerical_truth_table())
    print(feature_model.get_feature_branch('requestArrivalRate_50'))

{'101000': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0